### Fine-tune GPT

In [1]:
import json
import openai
import time, random
from datasets import load_dataset
print ("OpenAI package version:%s"%(openai.__version__))

OpenAI package version:0.28.0


### Load data from Hugging Face Datasets

In [2]:
dataset = load_dataset(path="AlekseyKorshuk/persona-chat",name="persona-chat")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [3]:
dataset["train"].to_pandas()

,personality,utterances
0,"[i like to remodel homes ., i like to go hunti...",[{'candidates': ['my mom was single with 3 boy...
1,"[my mom is my best friend ., i have four siste...",[{'candidates': ['there was one person better ...
2,"[i had a gig at local theater last night ., i ...",[{'candidates': ['fine how are you feeling ton...
3,"[i'm very athletic ., i wear contacts ., i hav...","[{'candidates': ['cool , i'm currently studyin..."
4,"[i am primarily a meat eater ., i am a guitar ...","[{'candidates': ['yes , i like the green bay p..."
...,...,...
17873,"[i sell miscellaneous stuff in local fairs ., ...","[{'candidates': ['interesting , luis is your g..."
17874,"[i currently work at mcdonalds ., i live with ...","[{'candidates': ['her initials are c . c .', '..."
17875,"[i've a daughter ., i'm under 6 feet tall ., i...",[{'candidates': ['ya . but not as unwelcomed a...
17876,"[i have a severe phobia of wide open spaces .,...",[{'candidates': ['i like to be smart . i'm ver...


### Since the nunber of training set is too small, we'll use test set

In [4]:
NUM_TRAIN      = 1000
NUM_VALIDATION = 100

In [5]:
base_text = """
You are a interactive agent that follows the given persona:
"""

In [6]:
train_jsonl_path = '../data/persona_chat_train.json1'
validation_jsonl_path  = '../data/persona_chat_validation.json1'
with open(train_jsonl_path, 'w') as f:
    for i in range(NUM_TRAIN):
        data = dataset['train'][i]
        persona =  " ".join(data['personality'])
        history = data['utterances'][-1]['history']
        message = [{"role": "system", "content": persona}]
        for i, h in enumerate(history):
            if (i%2 == 0):
                role = "user"
            else: role = "assistant"
            message.append({"role": role, "content": h})
        line = {"messages": message}
        # line = {"messages": [{"role": "system", "content": persona}, 
        #             {"role": "user", "content": "Question: " + data['question']},
        #             {"role": "assistant", "content": "Answer: " + data['answer']}]}
        f.write(json.dumps(line) + '\n')

with open(validation_jsonl_path, 'w') as f:
    for i in range(NUM_TRAIN,NUM_TRAIN+NUM_VALIDATION):
        data = dataset['train'][i]
        persona =  " ".join(data['personality'])
        history = data['utterances'][-1]['history']
        message = [{"role": "system", "content": persona}]
        for i, h in enumerate(history):
            if (i%2 == 0):
                role = "user"
            else: role = "assistant"
            message.append({"role": role, "content": h})
        line = {"messages": message}
        f.write(json.dumps(line) + '\n')
print ("[%s] saved."%(train_jsonl_path))
print ("[%s] saved."%(validation_jsonl_path))

[../data/persona_chat_train.json1] saved.
[../data/persona_chat_validation.json1] saved.


### Locate the key

In [7]:
key_path = '../key/rilab_key.txt'
print ('key_path:[%s]'%(key_path))

key_path:[../key/rilab_key.txt]


In [8]:
with open(key_path, 'r') as f: OPENAI_API_KEY = f.read()
openai.api_key = OPENAI_API_KEY

### Delete existing files (optional)

In [9]:
past_file_lists = openai.File.list()
for past_file in past_file_lists['data']:
    print (past_file)

{
  "object": "file",
  "id": "file-HW2ySBmxZGxbkavqN5CwFTgD",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 137019,
  "created_at": 1694411483,
  "status": "processed",
  "status_details": null
}
{
  "object": "file",
  "id": "file-rEaiBrRYmva7r571MumUT7xv",
  "purpose": "fine-tune-results",
  "filename": "step_metrics.csv",
  "bytes": 17983,
  "created_at": 1694413646,
  "status": "processed",
  "status_details": null
}
{
  "object": "file",
  "id": "file-0Wrj9UoWNcXGAFjrSmqPeCpA",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 1364436,
  "created_at": 1694411480,
  "status": "processed",
  "status_details": null
}


In [10]:
past_file_lists = openai.File.list()
for past_file in past_file_lists['data']:
    idx = past_file['id']
    if past_file['status'] == 'processed':
        print (past_file)
        openai.File.delete(idx)

{
  "object": "file",
  "id": "file-0Wrj9UoWNcXGAFjrSmqPeCpA",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 1364436,
  "created_at": 1694411480,
  "status": "processed",
  "status_details": null
}
{
  "object": "file",
  "id": "file-HW2ySBmxZGxbkavqN5CwFTgD",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 137019,
  "created_at": 1694411483,
  "status": "processed",
  "status_details": null
}
{
  "object": "file",
  "id": "file-rEaiBrRYmva7r571MumUT7xv",
  "purpose": "fine-tune-results",
  "filename": "step_metrics.csv",
  "bytes": 17983,
  "created_at": 1694413646,
  "status": "processed",
  "status_details": null
}


In [11]:
# Print after deleting files
past_file_lists = openai.File.list()
for past_file in past_file_lists['data']:
    print (past_file)

### Upload the dataset

In [12]:
file_train_data = openai.File.create(
  file=open(train_jsonl_path, "rb"),
  purpose='fine-tune'
)
file_train_data_id = file_train_data.id

In [13]:
file_test_data = openai.File.create(
  file=open(validation_jsonl_path, "rb"),
  purpose='fine-tune'
)
file_test_data_id = file_test_data.id

In [14]:
# Print after deleting files
past_file_lists = openai.File.list()
for past_file in past_file_lists['data']:
    print (past_file)

{
  "object": "file",
  "id": "file-fh77pphFD9aiezuGPntLPRxm",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 1364436,
  "created_at": 1694523224,
  "status": "uploaded",
  "status_details": null
}
{
  "object": "file",
  "id": "file-LNhNveOLUVZEdbhwLJKXe7u8",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 137019,
  "created_at": 1694523225,
  "status": "uploaded",
  "status_details": null
}


### Wait for your data to be processed

In [15]:
while True:
    files = openai.File.list()
    completed = True
    for f_idx,file in enumerate(files['data']):
        print(f_idx,file['id'], file['status'])
        if file['id'] == file_train_data_id or file['id'] == file_test_data_id:
            processed = (file['status'] == 'processed')
            completed = completed and processed
    if completed:
        break
    time.sleep(10)
print ("Ready.")

0 file-fh77pphFD9aiezuGPntLPRxm uploaded
1 file-LNhNveOLUVZEdbhwLJKXe7u8 uploaded
0 file-fh77pphFD9aiezuGPntLPRxm uploaded
1 file-LNhNveOLUVZEdbhwLJKXe7u8 uploaded
0 file-fh77pphFD9aiezuGPntLPRxm uploaded
1 file-LNhNveOLUVZEdbhwLJKXe7u8 uploaded
0 file-fh77pphFD9aiezuGPntLPRxm uploaded
1 file-LNhNveOLUVZEdbhwLJKXe7u8 uploaded
0 file-fh77pphFD9aiezuGPntLPRxm uploaded
1 file-LNhNveOLUVZEdbhwLJKXe7u8 processed
0 file-fh77pphFD9aiezuGPntLPRxm uploaded
1 file-LNhNveOLUVZEdbhwLJKXe7u8 processed
0 file-fh77pphFD9aiezuGPntLPRxm uploaded
1 file-LNhNveOLUVZEdbhwLJKXe7u8 processed
0 file-fh77pphFD9aiezuGPntLPRxm uploaded
1 file-LNhNveOLUVZEdbhwLJKXe7u8 processed
0 file-fh77pphFD9aiezuGPntLPRxm uploaded
1 file-LNhNveOLUVZEdbhwLJKXe7u8 processed
0 file-fh77pphFD9aiezuGPntLPRxm uploaded
1 file-LNhNveOLUVZEdbhwLJKXe7u8 processed
0 file-LNhNveOLUVZEdbhwLJKXe7u8 processed
1 file-fh77pphFD9aiezuGPntLPRxm uploaded
0 file-fh77pphFD9aiezuGPntLPRxm uploaded
1 file-LNhNveOLUVZEdbhwLJKXe7u8 processed
0 file-L

### Cancel running models (optional)

In [16]:
# List 10 fine-tuning jobs
jobs = openai.FineTuningJob.list(limit=10)
jobs = jobs['data']
for job in jobs:
    print(job['id'], job['status'])
    job_id = job['id']
    completed = job['status'] != 'running'
    if not completed:
        # Cancel a job
        openai.FineTuningJob.cancel(job_id)

ftjob-xTZABRXWvpiYHl4TAjBWz29L succeeded
ftjob-TxmbAGAFPoNgDwGdNr9kM7FJ succeeded
ftjob-o3N7MRwfHtLXgvyheaO7pEHr succeeded
ftjob-FyMhJGWK7LaGlu94NuAadINS cancelled
ftjob-QcdcVVyLjBabDRhESarNTCpm succeeded
ftjob-fhipvS2c2q7G3t4s7CLQGgDH succeeded
ftjob-gNALz0Stw06p7e0NeI0isSAO succeeded
ftjob-DAZc9L3aEKg1mNY2zFmpgdhv succeeded


### Start fine-tuning

In [17]:
current_job = openai.FineTuningJob.create(
    training_file=file_train_data_id, model="gpt-3.5-turbo",
    validation_file=file_test_data_id, hyperparameters={"n_epochs":1, })
print ("Ready.")

Ready.


In [18]:
# List 10 fine-tuning jobs
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x28e382220> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-8jBQV5vc2pmpUifBuRHhr8OY",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1694523358,
      "finished_at": null,
      "fine_tuned_model": null,
      "organization_id": "org-bT1bA6ExTcdQphsyv85F0j6z",
      "result_files": [],
      "status": "running",
      "validation_file": "file-LNhNveOLUVZEdbhwLJKXe7u8",
      "training_file": "file-fh77pphFD9aiezuGPntLPRxm",
      "hyperparameters": {
        "n_epochs": 1
      },
      "trained_tokens": null,
      "error": null
    },
    {
      "object": "fine_tuning.job",
      "id": "ftjob-xTZABRXWvpiYHl4TAjBWz29L",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1694411586,
      "finished_at": 1694413644,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:korea-university::7xUgH1aA",
      "organization_id": "org-bT1bA6ExTcdQphsyv85F0j6z",
      "result_files": [
        "file-rEaiBrRYm

### Retrieve information

In [19]:
current_job_id = current_job['id']
print  ("current_job_id:[%s]"%(current_job_id))

current_job_id:[ftjob-8jBQV5vc2pmpUifBuRHhr8OY]


In [20]:
# Retrieve the state of a fine-tune
openai.FineTuningJob.retrieve(current_job_id)

<FineTuningJob fine_tuning.job id=ftjob-8jBQV5vc2pmpUifBuRHhr8OY at 0x28e388220> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-8jBQV5vc2pmpUifBuRHhr8OY",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1694523358,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-bT1bA6ExTcdQphsyv85F0j6z",
  "result_files": [],
  "status": "running",
  "validation_file": "file-LNhNveOLUVZEdbhwLJKXe7u8",
  "training_file": "file-fh77pphFD9aiezuGPntLPRxm",
  "hyperparameters": {
    "n_epochs": 1
  },
  "trained_tokens": null,
  "error": null
}

### Wait for fine-tuning

In [ ]:
while True:
    job = openai.FineTuningJob.retrieve(current_job_id)
    completed = job['status'] != 'running'
    # List up to 10 events from a fine-tuning job
    events = openai.FineTuningJob.list_events(id=current_job_id, limit=2)
    for event in events['data']:
        print(event['message'])
    if completed:
        break
    time.sleep(60)
events = openai.FineTuningJob.list_events(id=current_job_id, limit=10)
for event in events['data']:
    print("=====================================")
    print(event['message'])
    print(event['data'])
print ("Ready.")

Fine tuning job started
Created fine-tuning job: ftjob-8jBQV5vc2pmpUifBuRHhr8OY
Fine tuning job started
Created fine-tuning job: ftjob-8jBQV5vc2pmpUifBuRHhr8OY
Fine tuning job started
Created fine-tuning job: ftjob-8jBQV5vc2pmpUifBuRHhr8OY
Fine tuning job started
Created fine-tuning job: ftjob-8jBQV5vc2pmpUifBuRHhr8OY
Fine tuning job started
Created fine-tuning job: ftjob-8jBQV5vc2pmpUifBuRHhr8OY
Fine tuning job started
Created fine-tuning job: ftjob-8jBQV5vc2pmpUifBuRHhr8OY
Fine tuning job started
Created fine-tuning job: ftjob-8jBQV5vc2pmpUifBuRHhr8OY
Fine tuning job started
Created fine-tuning job: ftjob-8jBQV5vc2pmpUifBuRHhr8OY
Fine tuning job started
Created fine-tuning job: ftjob-8jBQV5vc2pmpUifBuRHhr8OY
Fine tuning job started
Created fine-tuning job: ftjob-8jBQV5vc2pmpUifBuRHhr8OY
Fine tuning job started
Created fine-tuning job: ftjob-8jBQV5vc2pmpUifBuRHhr8OY
Fine tuning job started
Created fine-tuning job: ftjob-8jBQV5vc2pmpUifBuRHhr8OY
Step 100/1000: training loss=1.76
Fine t

### Inference

In [ ]:
model_id = openai.FineTuningJob.retrieve(current_job_id)['fine_tuned_model']

In [ ]:
start_idx = NUM_TRAIN + NUM_VALIDATION
end_idx = len(dataset['train'])
sample_idx = random.randint(start_idx, end_idx)
sample = dataset['train'][sample_idx]
print(sample)

In [ ]:
persona =  " ".join(sample['personality'])
history = sample['utterances'][-1]['history']
messages = [
    {"role": "system", "content": persona},
    {"role": "user", "content": history[0]},
    {"role": "assistant", "content": history[1]},
    {"role": "user", "content": history[2]},
]

In [ ]:
response = completion = openai.ChatCompletion.create(
  model=model_id,
  messages=messages
)

In [ ]:
answer = response['choices'][0]['message']['content']
print('Persona: ' + persona)
print('user: ' + history[0])
print('assistant: ' + history[1])
print('user: ' + history[2])
print('response: ' + answer)
print('Ground Truth: ' + history[3])

In [ ]:
persona =  "I am an extroverted person "

messages = [
    {"role": "system", "content": persona},
    {"role": "user", "content": "If you see someone waving their hand what will you do?"}
]

In [ ]:
response = completion = openai.ChatCompletion.create(
  model=model_id,
  messages=messages
)

In [ ]:
answer = response['choices'][0]['message']['content']
print('Persona: ' + persona)
print('response: ' + answer)

In [ ]:
response = completion = openai.ChatCompletion.create(
  model='gpt-4',
  messages=messages
)
answer = response['choices'][0]['message']['content']
print('Persona: ' + persona)
print('response: ' + answer)